In [1]:
# If workig in drive
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
from sklearn.model_selection import train_test_split

In [2]:
def load_raw():
    return (
        np.load(r'/Users/amanarya/Colab Notebooks/IDC410/Assignment_4_Speech_recognition_MLP_NN/DATA/dev.npy', encoding='bytes', allow_pickle = True),        
        np.load(r'/Users/amanarya/Colab Notebooks/IDC410/Assignment_4_Speech_recognition_MLP_NN/DATA/dev_labels.npy', encoding='bytes', allow_pickle = True)
    )
data, labels = load_raw()   #loads the feature numpy array onto data and the label file onto labels
print("Loaded")

Loaded


In [3]:
x_data, y_data = np.array([]), np.array([])                 #the data numpy array was of shape 1108(no. of files/matrices) x 338(no. of frames/vectors in each matrices) x 40
for i in range(len(data)):                                  #It was flattened into a 2d array of 669294(total no. of frames/vectors) x 40
    if i==0:                                                #The labels numpy array of 1108 x 338 was also flattened into a 1d array of 669294 elements
        x_data = data[0]                                    #The input array is now x_data and output array is y_data
        y_data = labels[0]
    else:
        x_data = np.concatenate((x_data, data[i]))
        y_data = np.concatenate((y_data, labels[i]))
print(x_data.shape, y_data.shape)

(669294, 40) (669294,)


In [4]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.1, random_state = 42)     #The x_data and y_data are divided into test and train sets in the ratio 9:1
print(x_train.shape, y_train.shape)                                                                         
print(x_test.shape, y_test.shape)

(602364, 40) (602364,)
(66930, 40) (66930,)


In [5]:
y_train = tf.keras.utils.to_categorical(y_train)            #The y_data is further modulated as a set of probabilities assigned to 138 categories
y_test = tf.keras.utils.to_categorical(y_test)              #For example, y_train value of 2 is converted into an array [0, 0, 1, 0 ... 0] of size 138 that now serves as output array
print(y_train.shape, y_test.shape)                                          

(602364, 138) (66930, 138)


In [6]:
graph = tf.Graph()

In [8]:
with graph.as_default():
  x = tf.placeholder(tf.float32, shape = [None, 40])                                                  #The input layer has 40 nodes and the output has 138 nodes
  y = tf.placeholder(tf.float32, shape = [None, 138])

  #layer 1, dim = 70
  layer_1 = tf.Variable(tf.truncated_normal([40, 60]), trainable = True, name = 'layer_1' )           #For modelling, 3 hidden layers (60, 80, 100 nodes) were used
  layer_1_out = tf.matmul(x, layer_1)
  layer_1_out = tf.sigmoid(layer_1_out)

  #layer 2, dim = 100
  layer_2 = tf.Variable(tf.truncated_normal([60, 80]), trainable = True, name = 'layer_2' )
  layer_2_out = tf.matmul(layer_1_out, layer_2)
  layer_2_out = tf.sigmoid(layer_2_out)

  layer_3 = tf.Variable(tf.truncated_normal([80, 100]), trainable = True, name = 'layer_3' )
  layer_3_out = tf.matmul(layer_2_out, layer_3)
  layer_3_out = tf.sigmoid(layer_3_out)

  #layer 3, dim = 138
  layer_4 = tf.Variable(tf.truncated_normal([100, 138]), trainable = True, name = 'layer_4' )
  layer_4_out = tf.matmul(layer_3_out, layer_4)
  print(layer_4_out.get_shape())
  print("Layers defined")
  
  loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = layer_4_out, name = "loss")    #Softmax function was used to categorise the output i.e. assign probabilities to the 138 nodes as per the input
  avg_loss = tf.reduce_mean(loss)                                                                       #The loss function used is mean squared error between predicted set of probabilities and y_train
  optimizer = tf.train.AdamOptimizer(0.01).minimize(avg_loss)                                           #Adam Optimizer is used with learning rate 0.01, which was found to be better suited for this data through hit and trial
  print("Loss and optimiser defined")

  #accuracy
  probs = tf.nn.softmax(layer_4_out)                                                                    #The set of predicted probabilities from the last layer
  preds = tf.argmax(probs, axis = -1)                                                                   #The node with the maximum predicted probability is taken as the predicted label/output for that particular input/frame/vector
  hyps = tf.argmax(y, axis = -1)                                                                        #The actual label for the given input/frame/vector
  corrects = tf.cast(tf.equal(hyps, preds), tf.float32)                                                 #The total no. of matches between predicted labels and actual labels
  acc = tf.reduce_mean(corrects)                                                                        #The percentage of matches scaled to 1 is defined as accuracy
  print("Accuracy computation nodes included")

  saver = tf.train.Saver(max_to_keep = 2)


(None, 138)
Layers defined
Loss and optimiser defined
Accuracy computation nodes included


In [9]:
total = len(x_train)                                                                #Size of training dataset is 6,02,364. The batch size is taken 1/100 th of it ~ 6000.
EPOCHS = 30                                                                         #30 iterations to be done by the optimiser for reducing loss
batch_size = 6000

with tf.Session(graph = graph) as sess:
  sess.run(tf.global_variables_initializer())
  for e in range(EPOCHS):
    e_loss = 0                                                                  #loss for the current iteration
    e_acc = 0                                                                   #accuracy for the current iteration
    start = 0                                                                   #To keep track of number of vectors of the training data set that have been analysed
    b_count = 0                                                                 #To keep track of No. of batches that were formed
    while start < total:
      end = min(start + batch_size, total)
      bx = x_train[start:end]
      by = y_train[start:end]
      data = {x:bx, y:by}
      b_loss,_,b_acc = sess.run([avg_loss, optimizer, acc], feed_dict = data)
      e_loss += b_loss
      e_acc += b_acc
      start = end
      b_count += 1
    e_loss /= b_count                                                             #loss of current iteration(epoch) averaged over no. of batches
    e_acc /= b_count
    saver.save(sess, 'weights/mlp')                                               #session saved for later use
    print("Epoch", e+1, "/", EPOCHS, "LOSS", e_loss, "Accuracy", e_acc)

2024-03-19 13:30:46.058849: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled


Epoch 1 / 30 LOSS 4.8717231679670885 Accuracy 0.10102034617436699
Epoch 2 / 30 LOSS 3.6642358161435267 Accuracy 0.15543614652487311
Epoch 3 / 30 LOSS 3.4562861730556675 Accuracy 0.17949894483726803
Epoch 4 / 30 LOSS 3.334621521505979 Accuracy 0.19409857498537195
Epoch 5 / 30 LOSS 3.266304983951078 Accuracy 0.20306036097578484
Epoch 6 / 30 LOSS 3.224069269576875 Accuracy 0.20888277062095037
Epoch 7 / 30 LOSS 3.1930894025481575 Accuracy 0.21223425614361716
Epoch 8 / 30 LOSS 3.166796658298757 Accuracy 0.21611301541918576
Epoch 9 / 30 LOSS 3.144581849032109 Accuracy 0.2197281496949715
Epoch 10 / 30 LOSS 3.1265610798750774 Accuracy 0.2220324090506771
Epoch 11 / 30 LOSS 3.111486050161985 Accuracy 0.22382411124682663
Epoch 12 / 30 LOSS 3.0961527777190256 Accuracy 0.22631472039340747
Epoch 13 / 30 LOSS 3.083475049179379 Accuracy 0.22861809158089139
Epoch 14 / 30 LOSS 3.0715384766607 Accuracy 0.23023918197296633
Epoch 15 / 30 LOSS 3.0610045702150552 Accuracy 0.23192183909439804
Epoch 16 / 30 LO

In [10]:
random_test_index = np.array(range(0, len(x_test)))                                                                 #a set of indices created to choose from test data

test_batch_x = x_test[random_test_index]
test_batch_y = y_test[random_test_index]

with tf.Session(graph = graph) as sess:
  saver.restore(sess, 'weights/mlp')                                                                            #restoring the saved session
  test_batch = {x:test_batch_x, y:test_batch_y}                                                                 #to pass the set of x_test and y_test data to the session
  prediction, hypothesis, probabilities, accuracy = sess.run([preds, hyps, probs, acc], feed_dict = test_batch) #to extract predicted labels, actual labels and accuracy feeding test data to the model
  
  data = {'ID':list(random_test_index), 'True Label':list(hypothesis), 'Predicted Label':list(prediction)}      #dataframe created with columns ID, True Label and Predicted label
  df = pd.DataFrame.from_dict(data)
  df.to_csv(r'/Users/amanarya/Colab Notebooks/IDC410/Assignment_4_Speech_recognition_MLP_NN/OUTPUT.csv', index = False)       #dataframe written into a csv file
  print(accuracy)

INFO:tensorflow:Restoring parameters from weights/mlp


0.24766174
